앞서 살펴본 단어 추출 방법 및 명사 추출 방법은 soynlp 에 모아서 구현해뒀습니다. 이를 이용하여 2016-10-20 뉴스에서 명사를 추출하는 실습과 단어 점수를 계산하는 코드를 실습해 봅니다.

In [1]:
import soynlp
print(soynlp.__version__)

0.0.493


## Corpus

In [2]:
from soynlp.utils import DoublespaceLineCorpus
from lovit_textmining_dataset.navernews_10days import get_news_paths

corpus_path = get_news_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

## WordExtractor (Cohesion score, Branching Entropy, Accessor Variety)

WordExtractor는 Cohesion score, Branching Entropy, Accessor Variety 등을 한번에 계산할 수 있도록 만들어둔 클래스입니다.

In [3]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    max_left_length=10,
    max_right_length=6,
    min_frequency=5
)
word_extractor.train(corpus)

training was done. used memory 0.741 Gbse memory 0.794 Gb


WordExtractor.word_scores()를 하면, 모든 L, R의 subwords에 대하여 앞서 함께 연습한 Cohesion score, Branching Entropy, Accessor Variety, frequency 등을 모두 계산하여 출력하도록 해두었습니다. 여기서 계산하는 Branching Entropy는 어절 간의 글자들도 고려한 수치입니다.

return type은 {word:namedtuple} 형식입니다. Python의 namedtuple 형식이기 때문에 .을 이용하여 해당 값을 손쉽게 가져올 수 있습니다. 

leftside_frequency는 해당 단어가 L에 등장한 횟수이며, rightside_frequency는 해당 단어가 R에 등장한 횟수입니다. 

In [4]:
scores = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 360721
all accessor variety was computed # words = 360721


In [5]:
scores['뉴스']

Scores(cohesion_forward=0.487322733132789, cohesion_backward=0.22771099423991986, left_branching_entropy=2.877143706774324, right_branching_entropy=3.128831672462708, left_accessor_variety=144, right_accessor_variety=215, leftside_frequency=11340, rightside_frequency=7274)

In [6]:
scores['뉴스'].cohesion_forward

0.487322733132789

원 논문에서는 띄어쓰기가 없다고 가정합니다. 하지만 한국어에는 일부 띄어쓰기가 있습니다. 예를 들어 아래 문장에서 `에트와` 사이에는 띄어쓰기가 있기 때문에 `에트와`는 단어가 될 수 없습니다. soynlp 에는 이러한 정보를 이용하여 한국어에 적합한 branching entropy 와 accessor variety 를 계산합니다.

```
음악중심에 트와이스가 나왔습니다
```

In [7]:
for word in ['트와이', '트와이스']:
    lbe = scores[word].left_branching_entropy
    rbe = scores[word].right_branching_entropy
    print(word, '(%.3f, %.3f)' % (lbe, rbe))

트와이 (2.994, -0.000)
트와이스 (2.994, 2.046)


Cohesion 만 계산하고 싶다면 아래의 함수를 이용할 수도 있습니다.

In [8]:
# {word:(cohesion_l, cohesion_r)}
cohesion_scores = word_extractor.all_cohesion_scores()
cohesion_scores['뉴스']

all cohesion probabilities was computed. # words = 223348


(0.487322733132789, 0.22771099423991986)

그 외에 braching entropy 나 accessor variety 만 계산하고 싶다면 아래의 함수를 이용합니다.

In [9]:
branching_entropy = word_extractor.all_branching_entropy()
accessor_variety = word_extractor.all_accessor_variety()

all branching entropies was computed # words = 360721
all accessor variety was computed # words = 360721


## Noun extraction

명사 추출기 버전이 3 개가 있는데, 모델을 계속 개선하며 이전 모델을 남겨둔 것입니다. v2 는 성능이 가장 많이 개선된 모델입니다.

In [10]:
from soynlp.noun import LRNounExtractor
from soynlp.noun import LRNounExtractor_v2
from soynlp.noun import NewsNounExtractor

# noun_extractor = LRNounExtractor()
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(corpus, min_noun_frequency=5)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 403896 from 223357 sents. mem=0.932 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4434442, mem=1.571 Gb
[Noun Extractor] batch prediction was completed for 119705 words
[Noun Extractor] checked compounds. discovered 70639 compounds
[Noun Extractor] postprocessing detaching_features : 32506 -> 32351
[Noun Extractor] postprocessing ignore_features : 32351 -> 32167
[Noun Extractor] postprocessing ignore_NJ : 32167 -> 31921
[Noun Extractor] 31921 nouns (70639 compounds) with min frequency=5
[Noun Extractor] flushing was done. mem=1.778 Gb                    
[Noun Extractor] 76.04 % eojeols are covered


In [11]:
for word in ['정부', '정부의', '알아', '알아냈', '트와이스', '아이디', '아이디어', '아이오아이']:
    print('{}: {}'.format(word, nouns.get(word, '')))

정부: NounScore(frequency=3251, score=0.9893238434163701)
정부의: 
알아: NounScore(frequency=104, score=0.32786885245901637)
알아냈: 
트와이스: NounScore(frequency=654, score=0.992831541218638)
아이디: NounScore(frequency=100, score=1.0)
아이디어: NounScore(frequency=251, score=1.0)
아이오아이: NounScore(frequency=250, score=1.0)


## Tokenization

띄어쓰기가 잘 된 데이터에서 명사나 어간 부분만 잘라내기 위해서는 L-Tokenizer 도 충분합니다.

In [12]:
from soynlp.tokenizer import LTokenizer,MaxScoreTokenizer, RegexTokenizer

cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
ltokenizer = LTokenizer(scores=cohesion_scores)

ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다')

['박근혜', '게이트', '에', '대한', '조사', '가', '시작', '되었습니다']

In [13]:
ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다', flatten=False)

[('박근혜', ''), ('게이트', '에'), ('대한', ''), ('조사', '가'), ('시작', '되었습니다')]

In [14]:
ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다', remove_r=True)

['박근혜', '게이트', '대한', '조사', '시작']

MaxScoreTokenizer 역시 단어 점수가 필요합니다.

In [15]:
maxscoretokenizer = MaxScoreTokenizer(scores=cohesion_scores)
maxscoretokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다'.replace(' ',''))

['박근혜', '게이트', '에', '대한', '조사', '가', '시작', '되었습니다']

In [16]:
maxscoretokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다'.replace(' ',''), flatten=False)

[[('박근혜', 0, 3, 0.33444481802269577, 3),
  ('게이트', 3, 6, 0.30753526965628375, 3),
  ('에', 6, 7, 0.0, 1),
  ('대한', 7, 9, 0.1611131928631136, 2),
  ('조사', 9, 11, 0.164128519137783, 2),
  ('가', 11, 12, 0.0, 1),
  ('시작', 12, 14, 0.1257767904844923, 2),
  ('되었습니다', 14, 19, 0.2762976357271788, 5)]]

패턴을 이용하여 기본적인 띄어쓰기도 할 수 있습니다.

In [19]:
regextokenizer = RegexTokenizer()
regextokenizer.tokenize('이것123abc유후')

['이것', '123', 'abc', '유후']

In [20]:
from soynlp.tokenizer import normalize

normalize('아닠ㅋㅋㅋㅋㅋㅋㅋ큐ㅠㅠㅠㅠㅠㅠ이럴수가흐규흐규흐규흐규', num_repeat=2)

'아니ㅋㅋㅠㅠ이럴수가흐규흐규'